In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.3)

import itertools as it
from collections import Counter, defaultdict, deque

from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, FunctionPolicy
from agents import *

In [67]:
# 2 by 2 env with 8-valued discrete rewards
env = OldMouselabEnv(2, 2, quantization=8, cost=0, ground_truth=False)
agent = Agent()
agent.register(env)

In [ ]:
# Exactly compute V(init)
from exact import solve
Q, V, pi, info = solve(env)
V(env.init)

In [64]:
# Empirically estimate V(init) for the exact optimal policy
agent.register(FunctionPolicy(pi))
df = pd.DataFrame(agent.run_many(5000))
df['return'].mean()

HBox(children=(IntProgress(value=0, max=5000), HTML(value='')))

1.4071110386514993

In [65]:
# Empirically estimate V(init) for full observation policy
# This policy is optimal when cost=0
agent.register(FixedPlanPolicy(range(5)))
df = pd.DataFrame(agent.run_many(5000))
df['return'].mean()

HBox(children=(IntProgress(value=0, max=5000), HTML(value='')))

1.4226388227938815